In [2]:
import numpy as np
import itertools
import tabulate
import pandas as pd

In [3]:
def polynomial_form(x, p):
    """Polynomial representation of the reduced state snapshot data."""
    return [x**degree for degree in range(2, p+1)]


In [4]:
import numpy as np
import itertools

# def generate_multi_indices(r, p):
#     # Define a helper function to check if a combination meets the criteria
#     def is_valid_combination(index):
#         non_zeros = np.count_nonzero(index)
#         return (sum(index) >= 3 and sum(index) <= 2*p) and non_zeros <= 2 and not ((max(index) > p) and non_zeros > 1)
    
#     # Generate and filter combinations on-the-fly
#     valid_indices = []
#     for combination in itertools.product(range(2*p+1), repeat=r):
#         if is_valid_combination(combination):
#             valid_indices.append(combination)
    
#     return valid_indices

def generate_multi_indices(r, p):
    def is_valid_combination(index):
        non_zeros = np.count_nonzero(index)
        return (sum(index) >= 3 and sum(index) <= 2*p) and non_zeros <= 2 and not ((max(index) > p) and non_zeros > 1)
    
    valid_indices = [combination for combination in itertools.product(range(2*p+1), repeat=r) if is_valid_combination(combination)]

    sorted_valid_indices = sorted(valid_indices, key=lambda x: sum(x))
    
    return sorted_valid_indices

# Test the function with smaller values to avoid crashing
r = 6  # Reduced number of elements
p = 4
multi_indices = generate_multi_indices(r, p)
print(f"Number of valid multi-indices: {len(multi_indices)}")
# Print a few examples
print("Some example multi-indices:")
for index in multi_indices[:5]:  # Print the first 5 examples
    print(index)


Number of valid multi-indices: 261
Some example multi-indices:
(0, 0, 0, 0, 0, 3)
(0, 0, 0, 0, 1, 2)
(0, 0, 0, 0, 2, 1)
(0, 0, 0, 0, 3, 0)
(0, 0, 0, 1, 0, 2)


In [5]:
# import itertools

# def generate_multi_indices(r, p):
#     # Generate all possible combinations of r elements with values between 0 and p
#     multi_indices = list(itertools.product(range(2*p+1), repeat=r))
    
#     # Filter out combinations where the sum of the elements is greater than p
#     # multi_indices = [index for index in multi_indices if ((sum(index) <= p))] # standard case
#     multi_indices = [index for index in multi_indices if ((sum(index) >= 3) and (sum(index) <= 2*p))]
#     # multi_indices = [index for index in multi_indices if not ((max(index) > 3) and (min(index) != 0))]
#     multi_indices = [index for index in multi_indices if not ((max(index) > p) and np.count_nonzero(index) > 1)]
    
#     # filter to retain only those multi_indices with 2 non-zero elements and 1 single non-zero element
#     multi_indices = [index for index in multi_indices if (np.count_nonzero(index) <= 2)]
    
#     return multi_indices

# # Test the function
# r = 10
# p = 4
# multi_indices = generate_multi_indices(r, p)
# print(multi_indices)
# print(len(multi_indices))

In [6]:
import numpy as np

def compute_polynomial_terms(x, multi_indices):
    """
    Compute polynomial terms for a vector x based on provided multi-indices.

    Args:
    x (np.ndarray): Input vector of shape (N,), where N is the number of elements.
    multi_indices (list of tuples): Multi-indices indicating the powers for each term.

    Returns:
    np.ndarray: A vector of computed polynomial terms, one for each multi-index.
    """
    # Pre-compute powers of x up to the maximum degree specified by multi_indices
    max_degree = max(max(index) for index in multi_indices)
    powers = np.ones((len(x), max_degree + 1))
    for degree in range(1, max_degree + 1):
        powers[:, degree] = x ** degree
    
    # Initialize the result vector to store the computed polynomial term for each multi-index
    result_vector = np.ones(len(multi_indices))
    
    for i, indices in enumerate(multi_indices):
        term = 1
        for var_index, power in enumerate(indices):
            if power > 0:
                term *= powers[var_index, power]
            else:
                term *= 1  # Multiply by 1 if the power is 0
        result_vector[i] = term  # Assuming you want the sum of terms across all samples
    
    return result_vector

# Example usage
x = np.array([1, 2])  # Example vector with 4 elements (e.g., x1, x2, x3, x4)
p = 2
multi_indices = generate_multi_indices(len(x), p)

# Compute polynomial terms
print("Generated multi-indices:")
print(tabulate.tabulate(multi_indices, headers=[f"x{i}" for i in range(1, len(x)+1)]))
polynomial_terms = compute_polynomial_terms(x, multi_indices)
print(polynomial_terms)


Generated multi-indices:
  x1    x2
----  ----
   0     3
   1     2
   2     1
   3     0
   0     4
   2     2
   4     0
[ 8.  4.  2.  1. 16.  4.  1.]


In [7]:
import numpy as np

def compute_polynomial_terms_2d_transposed(X, multi_indices):
    """
    Compute polynomial terms for a 2D array X transposed, where each row is a variable
    and each column a sample, based on provided multi_indices. The output is structured
    so that each row corresponds to a polynomial term and each column to a sample.

    Args:
    X (np.ndarray): Input 2D array of shape (N, M), where N is the number of variables and M is the number of samples.
    multi_indices (list of tuples): Multi-indices indicating the powers for each term.

    Returns:
    np.ndarray: A 2D array of computed polynomial terms, with shape (len(multi_indices), M),
                where each row corresponds to a polynomial term and each column to a sample.
    """
    N, M = X.shape
    max_degree = max(max(index) for index in multi_indices)
    
    # Pre-compute powers of each variable (row in X) up to the maximum degree
    powers = np.ones((N, max_degree + 1, M))
    for n in range(N):
        for degree in range(1, max_degree + 1):
            powers[n, degree, :] = X[n, :] ** degree

    # Initialize the result array to store the computed polynomial term for each multi-index
    result_array = np.ones((len(multi_indices), M))

    for i, indices in enumerate(multi_indices):
        term = np.ones(M)
        for n, power in enumerate(indices):
            if power > 0:
                term *= powers[n, power, :]
        result_array[i, :] = term

    return result_array

# Example usage
X = np.array([[1, 3, 5], [2, 4, 6], [3, 5, 7]])  # Example 2D array with 2 variables (N = 2) and 3 samples (M = 3)
p = 2
# Assuming generate_multi_indices is defined as before and suitable for the context
multi_indices = generate_multi_indices(X.shape[0], p)  

# Compute polynomial terms for 2D array X, with the specified output shape
polynomial_terms_2d_transposed = compute_polynomial_terms_2d_transposed(X, multi_indices)
print(polynomial_terms_2d_transposed)


[[2.700e+01 1.250e+02 3.430e+02]
 [1.800e+01 1.000e+02 2.940e+02]
 [1.200e+01 8.000e+01 2.520e+02]
 [8.000e+00 6.400e+01 2.160e+02]
 [9.000e+00 7.500e+01 2.450e+02]
 [4.000e+00 4.800e+01 1.800e+02]
 [3.000e+00 4.500e+01 1.750e+02]
 [2.000e+00 3.600e+01 1.500e+02]
 [1.000e+00 2.700e+01 1.250e+02]
 [8.100e+01 6.250e+02 2.401e+03]
 [3.600e+01 4.000e+02 1.764e+03]
 [1.600e+01 2.560e+02 1.296e+03]
 [9.000e+00 2.250e+02 1.225e+03]
 [4.000e+00 1.440e+02 9.000e+02]
 [1.000e+00 8.100e+01 6.250e+02]]


In [8]:
print(f"Generated {len(multi_indices)} multi-indices:")
multi_indices

Generated 15 multi-indices:


[(0, 0, 3),
 (0, 1, 2),
 (0, 2, 1),
 (0, 3, 0),
 (1, 0, 2),
 (1, 2, 0),
 (2, 0, 1),
 (2, 1, 0),
 (3, 0, 0),
 (0, 0, 4),
 (0, 2, 2),
 (0, 4, 0),
 (2, 0, 2),
 (2, 2, 0),
 (4, 0, 0)]

In [9]:
# print tabulated results without index column
print(tabulate.tabulate(pd.DataFrame(X), showindex=False))
print(tabulate.tabulate(pd.DataFrame(polynomial_terms_2d_transposed), showindex=False))

-  -  -
1  3  5
2  4  6
3  5  7
-  -  -
--  ---  ----
27  125   343
18  100   294
12   80   252
 8   64   216
 9   75   245
 4   48   180
 3   45   175
 2   36   150
 1   27   125
81  625  2401
36  400  1764
16  256  1296
 9  225  1225
 4  144   900
 1   81   625
--  ---  ----


In [8]:
from pyinstrument import Profiler
from OpInf import generate_multi_indices_efficient, monomial_degrees, gen_poly
import numpy as np
import numba as nb

# # @nb.njit(fastmath=True)
# def gen_poly(X, p, multi_indices=None):
#     """
#     Compute polynomial terms for a 2D array X transposed, where each row
#     is a variable and each column a sample, based on provided multiIndices.
#     The output is structured so that each row corresponds to a polynomial
#     term and each column to a sample.

#     Parameters:
#     X (np.ndarray): N-by-M data matrix where N is the number of variables, M is the number of samples.
#     p (int): Degree of the polynomial.

#     Returns:
#     resultArray (np.ndarray): Array containing the computed polynomial terms.
#     """
#     N, M = X.shape
#     # print("N: ", N)
#     # print("M: ", M)
#     if multi_indices is None:
#         raise ValueError("Multi-indices must be provided for this function")

#     # print("Generated {} multi-indices...".format(multi_indices.shape[0]))
#     # print("The multi-indices are: ", multi_indices)

#     max_degree = 2 * p

#     powers = np.ones((N, max_degree + 1, M), dtype=X.dtype)
#     for n in range(N):
#         for degree in range(1, max_degree + 1):
#             powers[n, degree, :] = X[n, :] ** degree

#     resultArray = np.ones((multi_indices.shape[0], M), dtype=X.dtype)
#     for i, indices in enumerate(multi_indices):
#         term = np.ones(M)
#         for n in range(N):
#             power = indices[n]  # Use directly for Python's zero-based indexing
#             if power > 0:  # Check if power is > 0, since 0th degree is already handled
#                 term *= powers[n, power, :]
#         resultArray[i, :] = term

#     return resultArray

p = 2
n_var = 50
n_timesteps = 10000
multi_indices = generate_multi_indices_efficient(n_var, p)


X = np.random.rand(n_var, n_timesteps)

profiler = Profiler()

try:
    profiler.stop()
except RuntimeError:
    pass

profiler.start()
poly = gen_poly(X, p, multi_indices)
profiler.stop()

print(profiler.output_text(unicode=True, color=True))


  _     ._   __/__   _ _  _  _ _/_   Recorded: 12:29:28  Samples:  2
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.109     CPU time: 0.109
/   _/                      v4.6.2

Program: /home/jy384/miniconda3/envs/fenicsx/lib/python3.11/site-packages/ipykernel_launcher.py --f=/home/jy384/.local/share/jupyter/runtime/kernel-v2-2784242boBaxynxw6ZN.json

0.109 <module>  ../../../../../tmp/ipykernel_2794813/2493173622.py:1




In [10]:
X = np.random.rand(n_var, n_timesteps)

profiler = Profiler()

try:
    profiler.stop()
except RuntimeError:
    pass

profiler.start()
poly = gen_poly(X, p, multi_indices)
profiler.stop()

print(profiler.output_text(unicode=True, color=True))


  _     ._   __/__   _ _  _  _ _/_   Recorded: 12:09:46  Samples:  2
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.290     CPU time: 0.140
/   _/                      v4.6.2

Program: /home/jy384/miniconda3/envs/fenicsx/lib/python3.11/site-packages/ipykernel_launcher.py --f=/home/jy384/.local/share/jupyter/runtime/kernel-v2-2784242Oj2tuvVd650E.json

0.289 <module>  ../../../../../tmp/ipykernel_2790200/4134463422.py:1




In [7]:
X = np.random.rand(n_var, n_timesteps)

profiler = Profiler()

try:
    profiler.stop()
except RuntimeError:
    pass

profiler.start()
poly = gen_poly(X, p, multi_indices)
profiler.stop()

print(profiler.output_text(unicode=True, color=True))


  _     ._   __/__   _ _  _  _ _/_   Recorded: 12:09:06  Samples:  2
 /_//_/// /_\ / //_// / //_'/ //     Duration: 1.898     CPU time: 12.306
/   _/                      v4.6.2

Program: /home/jy384/miniconda3/envs/fenicsx/lib/python3.11/site-packages/ipykernel_launcher.py --f=/home/jy384/.local/share/jupyter/runtime/kernel-v2-2784242Oj2tuvVd650E.json

1.898 <module>  ../../../../../tmp/ipykernel_2790200/4134463422.py:1




In [4]:
# import numpy as np

# @nb.njit(fastmath=True, parallel=True)
# def gen_poly_vec(X, p, multi_indices=None):
#     # N, M = X.shape
#     # if multi_indices is None:
#     #     raise ValueError("Multi-indices must be provided for this function")

#     # max_degree = 2 * p
    
#     X = X[:, np.newaxis, :]
#     multi_indices = multi_indices.T[:, :, np.newaxis]

#     resultArray = np.power(X, multi_indices).prod()
#     # resultArray = np.prod(np.power(X[:, np.newaxis, :], multi_indices), axis=0)

#     return resultArray

In [1]:

# profiler = Profiler()

# try:
#     profiler.stop()
# except RuntimeError:
#     pass

# # profiler.start()
# poly_vec = gen_poly_vec(X, p, multi_indices)
# profiler.stop()

# print(profiler.output_text(unicode=True, color=True))

In [12]:
# np.allclose(poly, poly_vec)

True

In [3]:
# profiler = Profiler()

# try:
#     profiler.stop()
# except RuntimeError:
#     pass

# profiler.start()
# poly = gen_poly_jax(X, p, multi_indices)
# profiler.stop()

In [4]:
# print(profiler.output_text(unicode=True, color=True))


  _     ._   __/__   _ _  _  _ _/_   Recorded: 23:06:19  Samples:  92
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.521     CPU time: 0.442
/   _/                      v4.6.2

Program: /home/jy384/miniconda3/envs/fenicsx/lib/python3.11/site-packages/ipykernel_launcher.py --f=/home/jy384/.local/share/jupyter/runtime/kernel-v2-2202794wNt1bA309SBf.json

0.520 <module>  ../../../../../tmp/ipykernel_2421558/2258413166.py:1
└─ 0.520 gen_poly_jax  OpInf.py:50
   └─ 0.520 vmap_f  jax/_src/api.py:1227
         [2 frames hidden]  jax
            0.520 WrappedFun.call_wrapped  jax/_src/linear_util.py:178
            ├─ 0.375 <lambda>  OpInf.py:70
            │  ├─ 0.347 arange  jax/_src/numpy/lax_numpy.py:2482
            │  │     [53 frames hidden]  jax, jaxlib, importlib
            │  │        0.194 batched_device_put  jax/_src/interpreters/pxla.py:169
            │  ├─ 0.022 power  jax/_src/numpy/ufuncs.py:328
            │  │     [20 frames hidden]  jax
            │  └─ 0.006 ArrayImpl.